In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [2]:
import pickle

with open('../../assets/credit.pkl', 'rb') as file:
  X_credit_train, y_credit_train, X_credit_test, y_credit_test = pickle.load(file)

In [4]:
X_credit_train.shape, y_credit_train.shape

((1500, 3), (1500,))

In [5]:
X_credit_test.shape, y_credit_test.shape

((500, 3), (500,))

In [6]:
import numpy as np

In [8]:
X_credit = np.concatenate((X_credit_train, X_credit_test), axis=0)
X_credit.shape

(2000, 3)

In [9]:
X_credit

array([[-1.3754462 ,  0.50630999,  0.10980934],
       [ 1.45826409, -1.64894017, -1.21501497],
       [-0.79356829,  0.22531104, -0.43370226],
       ...,
       [ 1.37445674, -1.05746369, -1.12564819],
       [-1.57087737, -0.6348826 , -0.36981671],
       [-1.03572293, -0.93978209,  0.04244312]])

In [11]:
y_credit = np.concatenate((y_credit_train, y_credit_test))
y_credit.shape

(2000,)

In [12]:
y_credit

array([0, 0, 0, ..., 0, 1, 1])

## Árvore de decisão

In [14]:
parameters = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

In [15]:
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=parameters)
grid_search.fit(X_credit, y_credit)

best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

best_parameters, best_score

({'criterion': 'gini',
  'min_samples_leaf': 1,
  'min_samples_split': 5,
  'splitter': 'best'},
 0.9835)